# Lista 06

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Equação de Condução de calor

$$\frac{\partial^2 T}{\partial x^2}+\frac{\partial^2 T}{\partial y^2}=-\frac{A_0}{k}=-40$$

com $0\leq x\leq 10$ e $0\leq y \leq 10$ e com condições de contorno $$T(0,y) = T(10,y) = 0$$ e $$ T(x,0) = T(x,10) = 0 $$

$$ k\frac{\partial T}{\partial x_i}+HT=HT_\infty $$

In [2]:
def cramer(n,N):
    for e in xrange(n.shape[0]):
        A = np.array([[1,n[e,0,0],n[e,0,1]],
                      [1,n[e,1,0],n[e,1,1]],
                      [1,n[e,2,0],n[e,2,1]]])
        detA = np.linalg.det(A)
        
        for i in xrange(3):
            a = np.copy(A)
            a[:,i] = np.array([1,0,0])
            b = np.copy(A)
            b[:,i] = np.array([0,1,0])
            c = np.copy(A)
            c[:,i] = np.array([0,0,1])
            
            N[e,0,i] = np.linalg.det(a) / detA
            N[e,1,i] = np.linalg.det(b) / detA
            N[e,2,i] = np.linalg.det(c) / detA

    return N

def derivada(N):
    dN = np.zeros(shape=(N.shape[0],2,3))
    for e in xrange(N.shape[0]):
        dN[e,0] = N[e,:,1]
        dN[e,1] = N[e,:,2]
    return dN

def rigidity(n,dN):
    K = np.zeros(shape=(dN.shape[0],3,3))
    for e in xrange(dN.shape[0]):
        A = np.array([[1,n[e,0,0],n[e,0,1]],
                      [1,n[e,1,0],n[e,1,1]],
                      [1,n[e,2,0],n[e,2,1]]])
        detA = np.linalg.det(A)
        
        for i in xrange(3):
            for j in xrange(3):
                aux1 = dN[e,0,j]*dN[e,0,i]
                aux2 = dN[e,1,j]*dN[e,1,i]
                K[e,i,j] = 0.5 * detA * (aux1+aux2)
    return K

def force(n,N,c):
    H = np.zeros(shape=(N.shape[0],3))
    for e in xrange(N.shape[0]):
        A = np.array([[1,n[e,0,0],n[e,0,1]],
                      [1,n[e,1,0],n[e,1,1]],
                      [1,n[e,2,0],n[e,2,1]]])
        detA = np.linalg.det(A)
        
        H[e,0] = detA * c * 0.5
        H[e,1] = detA * c * 0.5
        H[e,2] = detA * c * 0.5
    return H

In [292]:
nx, ny = 3, 3
N_eleme = 2 * (nx-1) * (ny-1)
N_nodes = 3 * N_eleme
N_quad = N_eleme / 2

xx = np.linspace(0,10,nx)
yy = np.linspace(0,10,ny)
x = np.tile(xx,ny)
y = np.tile(yy,nx)
n = np.zeros(shape=(N_eleme,3,2))

e = 0
for j in xrange(ny-1):
    for i in xrange(nx-1):
        n[e,0] = np.array([x[1+i],y[1+j]])
        n[e,1] = np.array([x[0+i],y[1+j]])
        n[e,2] = np.array([x[0+i],y[0+j]])
        n[e+1,0] = np.array([x[1+i],y[1+j]])
        n[e+1,1] = np.array([x[0+i],y[0+j]])
        n[e+1,2] = np.array([x[1+i],y[0+j]])
        e+=2
        
N = np.zeros(shape=(N_eleme,3,3))
N = cramer(n,N)
dN = derivada(N)

In [316]:
# Matriz das [POS]ições
POS = np.zeros(shape=(N_eleme+1,2))
for j in xrange(ny):
    for i in xrange(nx):
        POS[i+(j*nx),0] = xx[i]
        POS[i+(j*nx),1] = yy[j]
        
# Matriz de [CON]ectividade
CON = np.zeros(shape=(N_eleme,3))
for e in xrange(N_eleme):
    CON[e,0] = int(np.where((POS[:,0]==n[e,0,0])&(POS[:,1]==n[e,0,1]))[0][0])
    CON[e,1] = int(np.where((POS[:,0]==n[e,1,0])&(POS[:,1]==n[e,1,1]))[0][0])
    CON[e,2] = int(np.where((POS[:,0]==n[e,2,0])&(POS[:,1]==n[e,2,1]))[0][0])
    
# [PO]sição dos elementos de [F]ronteira
POF = np.zeros(shape=(((nx-1)*2)+((ny-1)*2),2))
POF[0,0] = xx[0]
POF[1:nx-1,0] = xx[1:nx-1]
POF[nx-1,0] = xx[-1]
POF[nx:nx+ny-2,0] = xx[-1]
POF[nx+ny-2,0] = xx[-1]
POF[nx+ny-1:nx+ny+nx-3,0] = xx[nx-2:0:-1]
POF[nx+ny+nx-3,0] = xx[0]
POF[nx+ny+nx-2:,0] = xx[0]

POF[0,1] = yy[0]
POF[1:nx-1,1] = yy[0]
POF[nx-1,1] = yy[0]
POF[nx:nx+ny-2,1] = yy[1:-1]
POF[nx+ny-2,1] = yy[-1]
POF[nx+ny-1:nx+ny+nx-3,1] = yy[-1]
POF[nx+ny+nx-3,1] = yy[-1]
POF[nx+ny+nx-2:,1] = yy[nx-2:0:-1]

# Matriz de [CO]nectividade entre os pontos de [F]ronteira
COF = np.zeros(shape=(((nx-1)*2)+((ny-1)*2),2))
for e in xrange(COF.shape[0]):
    cx = np.where((POS[:,0]==POF[e,0])&(POS[:,1]==POF[e,1]))[0][0]
    try:
        cy = np.where((POS[:,0]==POF[e+1,0])&(POS[:,1]==POF[e+1,1]))[0][0]
    except:
        cy = np.where((POS[:,0]==POF[0,0])&(POS[:,1]==POF[0,1]))[0][0]
    COF[e,0] = int(cx)
    COF[e,1] = int(cy)
    
# Matriz de Rigidez para os elementos de Fronteira [FRL]
FRL

In [317]:
COF

array([[0., 1.],
       [1., 2.],
       [2., 5.],
       [5., 8.],
       [8., 7.],
       [7., 6.],
       [6., 3.],
       [3., 0.]])

In [299]:
POF.shape

(8, 2)

In [300]:
POS

array([[ 0.,  0.],
       [ 5.,  0.],
       [10.,  0.],
       [ 0.,  5.],
       [ 5.,  5.],
       [10.,  5.],
       [ 0., 10.],
       [ 5., 10.],
       [10., 10.]])

# Formulação Fraca

$$ T^{(e)}(x,y)=\sum_{i=1}^{3}T(x^{(e)},y^{(e)})N_i^{(e)}(x,y) $$

In [4]:
A0 = 100.
k = 2.5

K = rigidity(n,dN)
H = force(n,N,-A0/k)

In [5]:
H

array([[-500., -500., -500.],
       [-500., -500., -500.],
       [-500., -500., -500.],
       [-500., -500., -500.],
       [-500., -500., -500.],
       [-500., -500., -500.],
       [-500., -500., -500.],
       [-500., -500., -500.]])

In [6]:
n

array([[[ 5.,  5.],
        [ 0.,  5.],
        [ 0.,  0.]],

       [[ 5.,  5.],
        [ 0.,  0.],
        [ 5.,  0.]],

       [[10.,  5.],
        [ 5.,  5.],
        [ 5.,  0.]],

       [[10.,  5.],
        [ 5.,  0.],
        [10.,  0.]],

       [[ 5., 10.],
        [ 0., 10.],
        [ 0.,  5.]],

       [[ 5., 10.],
        [ 0.,  5.],
        [ 5.,  5.]],

       [[10., 10.],
        [ 5., 10.],
        [ 5.,  5.]],

       [[10., 10.],
        [ 5.,  5.],
        [10.,  5.]]])

In [7]:
N

array([[[ 0. ,  0.2,  0. ],
        [ 0. , -0.2,  0.2],
        [ 1. ,  0. , -0.2]],

       [[ 0. ,  0. ,  0.2],
        [ 1. , -0.2,  0. ],
        [ 0. ,  0.2, -0.2]],

       [[-1. ,  0.2,  0. ],
        [ 1. , -0.2,  0.2],
        [ 1. ,  0. , -0.2]],

       [[ 0. ,  0. ,  0.2],
        [ 2. , -0.2,  0. ],
        [-1. ,  0.2, -0.2]],

       [[ 0. ,  0.2,  0. ],
        [-1. , -0.2,  0.2],
        [ 2. ,  0. , -0.2]],

       [[-1. ,  0. ,  0.2],
        [ 1. , -0.2,  0. ],
        [ 1. ,  0.2, -0.2]],

       [[-1. ,  0.2,  0. ],
        [ 0. , -0.2,  0.2],
        [ 2. ,  0. , -0.2]],

       [[-1. ,  0. ,  0.2],
        [ 2. , -0.2,  0. ],
        [ 0. ,  0.2, -0.2]]])

In [116]:
n

array([[[ 5.,  5.],
        [ 0.,  5.],
        [ 0.,  0.]],

       [[ 5.,  5.],
        [ 0.,  0.],
        [ 5.,  0.]],

       [[10.,  5.],
        [ 5.,  5.],
        [ 5.,  0.]],

       [[10.,  5.],
        [ 5.,  0.],
        [10.,  0.]],

       [[ 5., 10.],
        [ 0., 10.],
        [ 0.,  5.]],

       [[ 5., 10.],
        [ 0.,  5.],
        [ 5.,  5.]],

       [[10., 10.],
        [ 5., 10.],
        [ 5.,  5.]],

       [[10., 10.],
        [ 5.,  5.],
        [10.,  5.]]])

In [117]:
n[0,1]

array([0., 5.])

In [120]:
CON

array([[4., 3., 0.],
       [4., 0., 1.],
       [5., 4., 1.],
       [5., 1., 2.],
       [7., 6., 3.],
       [7., 3., 4.],
       [8., 7., 4.],
       [8., 4., 5.]])